## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [ ]:
import os
import pandas as pd
path = r'./kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split(os.path.sep)[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split(os.path.sep)[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split(os.path.sep)[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split(os.path.sep)[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split(os.path.sep)[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split(os.path.sep)[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

Переведём типы движений в числа

In [ ]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [ ]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [ ]:
import pandas as pd
from matplotlib import pylab as plt
from scipy.signal import find_peaks
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    if a == -1:
        st = 1250
    else:
        st = a
    if b == -1:
        end = -1250
    else:
        end = b
    if cut == False:
        st = 0
        end = -1
    print(name)
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1
    yabs = None
    newFrq, newYabs = None, None
    x_, y_ = None, None
    ind = []
    try:
        Fs = 250 # частота сбора данных

        Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

        y = Acc.values
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n / Fs
        frq = k / T # two sides frequency range
        frq = frq[:len(frq) // 2] # one side frequency range

        Y = np.fft.fft(y) / n # dft and normalization
        Y = Y[:n // 2]
        
        yabs = abs(Y)
        
        # fig, ax3 = plt.subplots(nrows=1, ncols=1)

        # ax3.plot(frq,yabs) # plotting the spectrum
        # ax3.set_xlim([0,30])
        # ax3.set_ylim([0,0.1])
        ind = find_peaks(yabs, height=[0.05, 1.5])[0]
    except ValueError:
        ind = [""]
        
    # Амплитуда
    acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    amplitude = acc.max() - acc.min()
    fur = 1 if len(ind) == 0 else 0
    print('The amplitude:', acc.max() - acc.min())
    print('The fur: ', fur)
    return amplitude, fur

Ampl = np.zeros(len(Data)) 
Fr = np.zeros(len(Data))

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Ampl[track_num], Fr[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib
track_0.csv
The amplitude: 1.2409331302405158
The fur:  0
track_1.csv
The amplitude: 1.9935598883036225
The fur:  0
track_10.csv
The amplitude: 7.802304392990605
The fur:  0
track_100.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


The amplitude: 0.34113272676839523
The fur:  1
track_101.csv
The amplitude: 0.4539765748076483
The fur:  1
track_102.csv
The amplitude: 2.2110203625988167
The fur:  0
track_103.csv
The amplitude: 3.1345378975461413
The fur:  0
track_104.csv
The amplitude: 1.5750179236901383
The fur:  0
track_105.csv
The amplitude: 2.5835429119223496
The fur:  1
track_106.csv
The amplitude: 0.6312015938792386
The fur:  1
track_107.csv
The amplitude: 2.124336238883378
The fur:  1
track_108.csv
The amplitude: 3.1243961620554765
The fur:  0
track_109.csv
The amplitude: nan
The fur:  0
track_11.csv
The amplitude: 1.4499291676896982
The fur:  0
track_110.csv
The amplitude: 1.8330376632404348
The fur:  0
track_111.csv
The amplitude: 0.042436878839561376
The fur:  1
track_112.csv
The amplitude: 1.4825475120699383
The fur:  0
track_113.csv
The amplitude: 0.7902169184700258
The fur:  1
track_114.csv
The amplitude: 0.8574945808044203
The fur:  0
track_115.csv
The amplitude: 5.0103722896175285
The fur:  0
track_11

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [ ]:
def get_prediction(amplitude, fr):
    if amplitude < 1:
        return 0 # стояние
    if amplitude >  5:
        return 2 # бег
    if fr == 0:
        return 1 # велик
    return 3 # ходьба

In [ ]:
len(Data), len(Ampl), len(Target)

(146, 146, 146)

In [ ]:
type(Data)

list

In [ ]:
Ampl_new = []

i=0
for elem in Ampl:
    if not (math.isnan(elem) or elem == -1):
        i+=1
        Ampl_new.append(elem)
    else:
        Ampl_new.append(2)
len(Ampl_new), i

(146, 140)

Создадим таблицу, содержащую номер трека и предсказание.

In [ ]:
prediction = [get_prediction(a, b) for a, b in zip(Ampl, Fr)]

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,1
2,2,3
3,3,2
4,4,1
5,5,1
6,6,1
7,7,2
8,8,0
9,9,0


In [ ]:
Res.to_csv("5_submit.csv", index=False)